In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
import pandas as pd
from os import walk
import os
mypath = os.path.abspath(os.getcwd()) + "/filtered_genomes"

files = []
for (dirpath, dirnames, filenames) in walk(mypath):
    files.extend(filenames)
    break

ext = "embl"

In [2]:
FileNames = []
IntergenomicGC = []
Leu4Per = []
Leu2Per = []
Ser4Per = []
Ser2Per = []
Arg2Per = []
Arg4Per = []

files_tot = len(files)
for index, name in enumerate(files):
    print("\n")
    print(f"Analyzing file {index+1} of {files_tot}: {name}")
    print("\n")
    record = SeqIO.read(f'filtered_genomes/{name}', ext)
    sequence = record.seq
    FileNames.append(str(name))
    
    start = 0
    coding_sequences = []
    intergenomic_sequence = []  # It will contain the sequences out of the locations bounds
    intergenomic_sequence_list = []  # It will contain the locations bounds
    intergenomic_sequence_string = ""  # intergenomic sequence
    
    for feature in record.features:

        locations= feature.location
        s = int(locations.nofuzzy_start)
        e = int(locations.nofuzzy_end)
        st = int(locations.strand)
        
        gene_start = int(locations.nofuzzy_start)

        if gene_start > start:
            ig_sequence = sequence[start:gene_start]
            intergenomic_sequence.append(str(ig_sequence))
            intergenomic_sequence_list.append((start, gene_start))

        new_start = int(locations.nofuzzy_end)

        if new_start != len(sequence):
            start = new_start

        #Reversing complementary strands
        if st == -1:
            sequence_negativestrand = str(sequence[s:e])
            sequence_negativestrand= sequence_negativestrand[::-1]
            sequence_negativestrand= sequence_negativestrand.replace("A","b")
            sequence_negativestrand= sequence_negativestrand.replace("T","A")
            sequence_negativestrand= sequence_negativestrand.replace("b","T")
            sequence_negativestrand= sequence_negativestrand.replace("C","d")
            sequence_negativestrand= sequence_negativestrand.replace("G","C")
            sequence_negativestrand= sequence_negativestrand.replace("d","G")

        #Discarding the pseudogenes

            ok = 0
            genes = str(sequence_negativestrand)


            if len(genes)%3 != 0:
                ok= 1
            elif genes[:3] != "ATG" and genes[:3] != "GTG" and genes[:3] != "TTG":
                ok= 2
            elif genes[-3:] != "TAA" and genes[-3:] != "TAG" and genes[-3:] != "TGA":
                ok= 3


            if st == -1 and ok == 0:
                coding_sequences.append(genes)
        
        ok = 0
        genes = str(sequence[s:e])
    
        if len(genes)%3 != 0:
            ok= 1
        elif genes[:3] != "ATG" and genes[:3] != "GTG" and genes[:3] != "TTG":
            ok= 2
        elif genes[-3:] != "TAA" and genes[-3:] != "TAG" and genes[-3:] != "TGA":
            ok= 3
        elif len(genes) == len(sequence):
            ok= 4

        if st == 1 and ok == 0:
            coding_sequences.append(genes)

        #checking for errors
        check = False
        if check and st == 1 and ok != 0:
            print(locations)
            print(ok)
            print(genes[:3])
            print(genes[-3:])
            
    if start != len(sequence):
        ig_sequence = sequence[start:len(sequence)]
        intergenomic_sequence.append(str(ig_sequence))
        intergenomic_sequence_list.append((start, gene_start))

    intergenomic_sequence_string = "".join(intergenomic_sequence)

    # Intergenomic GC content
    IgGC = intergenomic_sequence_string.count("G") + intergenomic_sequence_string.count("C")
    IgGC_content = (IgGC / len(intergenomic_sequence_string)) * 100
    print(f"Intergenomic GC content: {IgGC_content:.2f}%")
    IntergenomicGC.append(IgGC_content)
    
    coding_sequencewhole = "".join(coding_sequences)
    aminoacids_sequence = Seq(coding_sequencewhole).translate()
    
    # Codon usage for Arginine, Leucine and Serine
    CGAcount = 0
    CGCcount = 0
    CGGcount = 0
    CGTcount = 0
    AGAcount = 0
    AGGcount = 0
    CTAcount = 0
    CTCcount = 0
    CTGcount = 0
    CTTcount = 0
    TTAcount = 0
    TTGcount = 0
    TCAcount = 0
    TCCcount = 0
    TCGcount = 0
    TCTcount = 0
    AGTcount = 0
    AGCcount = 0
    for cds in coding_sequences:
        for i in range(0, len(cds), 3):
            codons = cds[i: i + 3]
            #Codon usage for Arginine
            CGA = codons.count("CGA")
            CGAcount += CGA
            CGC = codons.count("CGC")
            CGCcount += CGC
            CGG = codons.count("CGG")
            CGGcount += CGG
            CGT = codons.count("CGT")
            CGTcount += CGT
            AGA = codons.count("AGA")
            AGAcount += AGA
            AGG = codons.count("AGG")
            AGGcount += AGG
            #Codon usage for Leucine
            CTA = codons.count("CTA")
            CTAcount += CTA
            CTC = codons.count("CTC")
            CTCcount += CTC
            CTG = codons.count("CTG")
            CTGcount += CTG
            CTT = codons.count("CTT")
            CTTcount += CTT
            TTA = codons.count("TTA")
            TTAcount += TTA
            TTG = codons.count("TTG")
            TTGcount += TTG
            #Codon usage for Serine
            TCA = codons.count("TCA")
            TCAcount += TCA
            TCC = codons.count("TCC")
            TCCcount += TCC
            TCG = codons.count("TCG")
            TCGcount += TCG
            TCT = codons.count("TCT")
            TCTcount += TCT
            AGT = codons.count("AGT")
            AGTcount += AGT
            AGC = codons.count("AGC")
            AGCcount += AGC
    Arginine4Total = CGGcount + CGAcount + CGCcount + CGTcount
    Arginine2Total = AGAcount + AGGcount
    Arg4Per.append(Arginine4Total/len(aminoacids_sequence)*100)
    Arg2Per.append(Arginine2Total/len(aminoacids_sequence)*100)
    Leucine4Total = CTGcount + CTAcount + CTCcount + CTTcount
    Leucine2Total = TTAcount + TTGcount
    Leu4Per.append(Leucine4Total/len(aminoacids_sequence)*100)
    Leu2Per.append(Leucine2Total/len(aminoacids_sequence)*100)
    Serine4Total = TCGcount + TCAcount + TCCcount + TCTcount
    Serine2Total = AGTcount + AGCcount
    Ser4Per.append(Serine4Total/len(aminoacids_sequence)*100)
    Ser2Per.append(Serine2Total/len(aminoacids_sequence)*100)
    



Analyzing file 1 of 645: AE000657.embl


Intergenomic GC content: 37.10%


Analyzing file 2 of 645: AE014184.embl


Intergenomic GC content: 44.71%


Analyzing file 3 of 645: AE015928.embl


Intergenomic GC content: 33.14%


Analyzing file 4 of 645: AE016825.embl


Intergenomic GC content: 58.22%


Analyzing file 5 of 645: AE017282.embl


Intergenomic GC content: 59.95%


Analyzing file 6 of 645: AL935263.embl


Intergenomic GC content: 37.56%


Analyzing file 7 of 645: AM180252.embl


Intergenomic GC content: 25.69%


Analyzing file 8 of 645: AM236080.embl


Intergenomic GC content: 55.00%


Analyzing file 9 of 645: AM260479.embl


Intergenomic GC content: 63.23%


Analyzing file 10 of 645: AM260525.embl


Intergenomic GC content: 33.09%


Analyzing file 11 of 645: AM420293.embl


Intergenomic GC content: 70.79%


Analyzing file 12 of 645: AM494475.embl


Intergenomic GC content: 26.70%


Analyzing file 13 of 645: AM711867.embl


Intergenomic GC content: 71.74%


Analyzing file 14 o



Analyzing file 109 of 645: CP000267.embl


Intergenomic GC content: 56.36%


Analyzing file 110 of 645: CP000282.embl


Intergenomic GC content: 40.81%


Analyzing file 111 of 645: CP000284.embl


Intergenomic GC content: 47.16%


Analyzing file 112 of 645: CP000285.embl


Intergenomic GC content: 61.80%


Analyzing file 113 of 645: CP000316.embl


Intergenomic GC content: 59.58%


Analyzing file 114 of 645: CP000319.embl


Intergenomic GC content: 58.90%


Analyzing file 115 of 645: CP000352.embl


Intergenomic GC content: 59.03%


Analyzing file 116 of 645: CP000360.embl


Intergenomic GC content: 56.86%


Analyzing file 117 of 645: CP000377.embl


Intergenomic GC content: 57.12%


Analyzing file 118 of 645: CP000383.embl


Intergenomic GC content: 31.14%


Analyzing file 119 of 645: CP000386.embl


Intergenomic GC content: 67.91%


Analyzing file 120 of 645: CP000387.embl


Intergenomic GC content: 33.40%


Analyzing file 121 of 645: CP000388.embl


Intergenomic GC content: 42.10%

Intergenomic GC content: 44.91%


Analyzing file 216 of 645: CP001618.embl


Intergenomic GC content: 72.31%


Analyzing file 217 of 645: CP001619.embl


Intergenomic GC content: 48.77%


Analyzing file 218 of 645: CP001628.embl


Intergenomic GC content: 72.38%


Analyzing file 219 of 645: CP001629.embl


Intergenomic GC content: 54.21%


Analyzing file 220 of 645: CP001630.embl


Intergenomic GC content: 72.16%


Analyzing file 221 of 645: CP001631.embl


Intergenomic GC content: 67.78%


Analyzing file 222 of 645: CP001632.embl


Intergenomic GC content: 36.37%


Analyzing file 223 of 645: CP001634.embl


Intergenomic GC content: 40.94%


Analyzing file 224 of 645: CP001643.embl


Intergenomic GC content: 72.32%


Analyzing file 225 of 645: CP001650.embl


Intergenomic GC content: 29.64%


Analyzing file 226 of 645: CP001673.embl


Intergenomic GC content: 34.38%


Analyzing file 227 of 645: CP001674.embl


Intergenomic GC content: 51.49%


Analyzing file 228 of 645: CP001678.embl





Analyzing file 322 of 645: CP002304.embl


Intergenomic GC content: 29.04%


Analyzing file 323 of 645: CP002305.embl


Intergenomic GC content: 39.06%


Analyzing file 324 of 645: CP002339.embl


Intergenomic GC content: 37.95%


Analyzing file 325 of 645: CP002343.embl


Intergenomic GC content: 69.99%


Analyzing file 326 of 645: CP002344.embl


Intergenomic GC content: 72.88%


Analyzing file 327 of 645: CP002345.embl


Intergenomic GC content: 36.31%


Analyzing file 328 of 645: CP002347.embl


Intergenomic GC content: 33.56%


Analyzing file 329 of 645: CP002349.embl


Intergenomic GC content: 33.99%


Analyzing file 330 of 645: CP002353.embl


Intergenomic GC content: 61.24%


Analyzing file 331 of 645: CP002355.embl


Intergenomic GC content: 42.38%


Analyzing file 332 of 645: CP002360.embl


Intergenomic GC content: 40.33%


Analyzing file 333 of 645: CP002361.embl


Intergenomic GC content: 68.55%


Analyzing file 334 of 645: CP002364.embl


Intergenomic GC content: 56.86%

Intergenomic GC content: 35.49%


Analyzing file 429 of 645: CP003350.embl


Intergenomic GC content: 62.74%


Analyzing file 430 of 645: CP003359.embl


Intergenomic GC content: 30.40%


Analyzing file 431 of 645: CP003360.embl


Intergenomic GC content: 48.12%


Analyzing file 432 of 645: CP003364.embl


Intergenomic GC content: 60.24%


Analyzing file 433 of 645: CP003379.embl


Intergenomic GC content: 58.96%


Analyzing file 434 of 645: CP003382.embl


Intergenomic GC content: 61.75%


Analyzing file 435 of 645: CP003389.embl


Intergenomic GC content: 67.09%


Analyzing file 436 of 645: CP003390.embl


Intergenomic GC content: 42.29%


Analyzing file 437 of 645: CP003403.embl


Intergenomic GC content: 43.47%


Analyzing file 438 of 645: CP003415.embl


Intergenomic GC content: 42.24%


Analyzing file 439 of 645: CP003418.embl


Intergenomic GC content: 28.56%


Analyzing file 440 of 645: CP003422.embl


Intergenomic GC content: 52.20%


Analyzing file 441 of 645: CP003466.embl





Analyzing file 535 of 645: CP007051.embl


Intergenomic GC content: 27.49%


Analyzing file 536 of 645: CP007053.embl


Intergenomic GC content: 56.41%


Analyzing file 537 of 645: CP007062.embl


Intergenomic GC content: 22.40%


Analyzing file 538 of 645: CP007128.embl


Intergenomic GC content: 69.58%


Analyzing file 539 of 645: CP007139.embl


Intergenomic GC content: 57.05%


Analyzing file 540 of 645: CP007142.embl


Intergenomic GC content: 41.97%


Analyzing file 541 of 645: CP007152.embl


Intergenomic GC content: 51.01%


Analyzing file 542 of 645: CP007155.embl


Intergenomic GC content: 65.98%


Analyzing file 543 of 645: CP007161.embl


Intergenomic GC content: 34.28%


Analyzing file 544 of 645: CP007201.embl


Intergenomic GC content: 32.61%


Analyzing file 545 of 645: CP007202.embl


Intergenomic GC content: 24.81%


Analyzing file 546 of 645: CP007444.embl


Intergenomic GC content: 61.52%


Analyzing file 547 of 645: CP007445.embl


Intergenomic GC content: 25.78%

Intergenomic GC content: 39.78%


Analyzing file 642 of 645: HG938353.embl


Intergenomic GC content: 54.86%


Analyzing file 643 of 645: LK028559.embl


Intergenomic GC content: 27.48%


Analyzing file 644 of 645: LN614827.embl


Intergenomic GC content: 35.37%


Analyzing file 645 of 645: LN614830.embl


Intergenomic GC content: 35.87%


In [3]:
df = pd.DataFrame(
    data=list(zip(
        FileNames,
        IntergenomicGC,
        Leu2Per,
        Leu4Per,
        Ser2Per,
        Ser4Per,
        Arg2Per,
        Arg4Per)
    ),
    columns=[
        "Name",
        "IntergenomicGC",
        "Leu2Per",
        "Leu4Per",
        "Ser2Per",
        "Ser4Per",
        "Arg2Per",
        "Arg4Per"
    ],
    index=range(1, files_tot+1)
)

folder = "results/"
print(f"Saving data to the {folder} folder")
df.to_csv(f"{folder}data.csv", index=False)

Saving data to the results/ folder
